<a href="https://colab.research.google.com/github/yuchen170/Mediapipe-Video-Processing/blob/main/Mediapipe_pose_landmark_and_bg_removal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 15.0 MB/s eta 0:00:00


In [ ]:
import cv2
import mediapipe as mp
import csv
import numpy as np
import os
from google.colab.patches import cv2_imshow

def write_landmarks_to_csv(landmarks, frame_number, csv_data):
    #print(f"Landmark coordinates for frame {frame_number}:")
    for idx, landmark in enumerate(landmarks):
        #print(f"{mp_pose.PoseLandmark(idx).name}: (x: {landmark.x}, y: {landmark.y}, z: {landmark.z})")
        csv_data.append([frame_number, mp_pose.PoseLandmark(idx).name, landmark.x, landmark.y, landmark.z])
    #print("\n")

video_path = '/content/drive/MyDrive/clip_single.mp4'
output_csv = '/content/drive/MyDrive/outputOfClip'
landmarks_folder = 'landmarks'
remove_bg_folder = 'remove_background'
if not os.path.exists(landmarks_folder):
    os.makedirs(landmarks_folder)
if not os.path.exists(remove_bg_folder):
    os.makedirs(remove_bg_folder)
# Initialize MediaPipe Pose and Drawing utilities
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

#pose = mp_pose.Pose()
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=2,
    enable_segmentation=True,
    min_detection_confidence=0.1,
    smooth_landmarks=True
)

# Open the video file
cap = cv2.VideoCapture(video_path)

frame_number = 0
csv_data = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Create a black background image
    black_background = np.zeros_like(frame_rgb)

    # Process the frame with MediaPipe Pose
    result = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Draw the pose landmarks on the black background
    if result.pose_landmarks:
        mp_drawing.draw_landmarks(black_background, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Add the landmark coordinates to the list and print them
        write_landmarks_to_csv(result.pose_landmarks.landmark, frame_number, csv_data)

        #save the frame
        landmarks_image_path = os.path.join(landmarks_folder, f'landmarks_{frame_number:06d}.png')
        cv2.imwrite(landmarks_image_path, black_background)
    #frame_height, frame_width, _ = frame.shape

    removed = frame.copy()
    #if result_bg.segmentation_mask is not None:
    condition = np.stack((result.segmentation_mask,) * 3, axis=-1) > 0.1
    bg_image = np.zeros(frame.shape, dtype=np.uint8)
    removed = np.where(condition, frame, bg_image)

    # Save background removed image
    bg_removed_image_path = os.path.join(remove_bg_folder, f'removeBG_{frame_number:06d}.png')
    cv2.imwrite(bg_removed_image_path, removed)

    # Display the combined frame
    #print(frame_number)
    #combined_frame = np.concatenate((removed, black_background), axis=1)
    #cv2_imshow(combined_frame)

    # Exit if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_number += 1

cap.release()

# Save the CSV data to a file
with open(output_csv, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['frame_number', 'landmark', 'x', 'y', 'z'])
    csv_writer.writerows(csv_data)
